# Desafio: Consumo de Dados para Previsão do Tempo das Cidades do Vale do Paraíba.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Neste desafio, você desenvolverá um notebook que será responsável por extrair dados de previsão do tempo das cidades do Vale do Paraíba, região onde se localiza a Dataside. Para consultar todas as cidades dessa região, utilizaremos a API do IBGE. No caso, basta realizar uma requisição HTTP com o método GET, utilizando a URL abaixo:

```
https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios
```

Com esses dados, gerar um data frame e a partir dele uma temp view. Ex: "cities"

Utilizando os nomes das cidades, deverão ser consultados os dados de previsão de tempo para cada cidade. Para realizar essa consulta, poderá ser utilizada qualquer uma das APIs informadas no link abaixo.

[Public APIs - Wather](https://github.com/public-apis/public-apis#weather)

Obs.: Para algumas, pode ser necessário cadastrar-se para acessar sua API Key. Mas nenhuma delas deve precisar cadastrar cartão de crédito ou adicionar qualquer valor monetário para utilizar. Caso alguma solicite, basta optar por outra.

Com os dados consultados, gerar um data frame e partir dele outra temp view. Ex: "forecasts"

Com as temp views geradas, utilizar Spark SQL para criar queries e gerar data frames das seguintes tabelas:

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento
    
    Obs.: Os valores da coluna "VaiChover" deverá ser "Sim" ou "Não". E a coluna "CodigoDaCidade" é o ID retornado junto com os nomes da cidades na API do IBGE.
    Obs.: Dependendo da API utilizada, algumas colunas podem não existir e ficarão em branco. Você deve optar por uma API que traga o maior número de informações possível.

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

Essas tabelas deverão ser exportadas em formado CSV e entregue no final do desafio.

## To Do

[ ] - Consultar municípios do Vale do Paraíba, gerar um data frame e criar uma temp view com esses dados.
[ ] - Consultar dados do tempo para cada município, gerar um data frame e criar uma outra temp view.
[ ] - Utilizar Spark SQL para gerar os data frames das Tabelas 1 e 2.
[ ] - Exportar os data frames para CSV.

## Atenção

- Existe um limite de requisições de 10000 requests por conta cadastrada na m3o.
- Essa API pode retornar cidades de outras regiões que possuem nome semelhante a alguma cidade do Vale do Paraiba. Pode mantê-las ou filtrar para gerar as tabelas apenas com dados de Regiao = Sao Paulo. Fica a seu critério.

## Entregando o desafio

Concluindo todos os passos informados em To Do, basta salvar o arquivo .ipynb do notebook e enviar para a Dataside juntamente com os CSVs das duas tabelas.


In [1]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

import requests
import json
# import unidecode
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

In [407]:
# Buscar cidades do Vale do Paraíba
url = "https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios"
response = requests.get(url)

# Criar Data Frame
citiesresponse_rdd = spark.sparkContext.parallelize([response.text])
citiesresponse_df = spark.read.json(citiesresponse_rdd)
citiesresponse =  citiesresponse_df.select('id', 'nome')

cidades = []
def func(row):
  txt = (row['nome'])
  transTable = txt.maketrans("áíç", "aic")
  cidade_nome = txt.translate(transTable)
  cidade_id = (row['id'])
  cidades.append({'nome': cidade_nome, 'id': cidade_id})
  return cidades

citiesCollect = citiesresponse.collect()
for row in citiesCollect:
    func(row)
    
cities_rdd = spark.sparkContext.parallelize(cidades)
cities = spark.read.json(cities_rdd)

# Criar view com as cidades
cities.createOrReplaceTempView('citiesView')

In [415]:
# Buscar previsão do tempo para as cidades

# WEATHERAPI.COM:
# OBS1: NÃO ENCONTRA A CIDADE DE POTIM ATRAVÉS DO NOME RETORNANDO NULL. UMA SOLUÇÃO SERIA REALIZAR A REQUISIÇÃO ATRAVÉS DO ID DA CIDADE
# POREM, A API NÃO POSSUI OPÇÃO DE QUERY COM O ID DA CIDADE.
# OBS2: O PARÂMETRO DAYS É RESPONSÁVEL PELO NÚMERO DE DIAS QUE DESEJA-SE SABER A PREVISÃO DO TEMPO. NA REQUISIÇÃO REALIZADA MANTEVE-SE DAYS = 5 (CONFORME ENUNCIADO DO DESAFIO),
# PORÉM, POR ESTAR UTILIZANDO UM PLANO FREE, O MÁXIMO DE DIAS A SEREM CONSULTADOS PERMITIDO PELA API É DE TRÊS DIAS.

data = []
def func(cidade_row):
  cidade = cidade_row.nome
  url_weather_api = f"http://api.weatherapi.com/v1/forecast.json?key=9f9593d03d174ef7946140457220605&q={cidade}&days=5&aqi=no&alerts=nob"
  response = requests.get(url_weather_api)
  data.append(response.text)
    
citiesCollect = cities.select('nome').collect()
for row in citiesCollect:
    func(row)

# Criar data frame com as cidades
rdd = spark.sparkContext.parallelize(data)
forecasts = spark.read.json(rdd)

# Criar view com as cidades
forecasts.createOrReplaceTempView('forecastsView')

In [416]:
# Criar DF da Tabela 1
DF_tabela1 = spark.sql("""
  SELECT
    Cidade, 
    CidadeID, 
    ForecastDay.date as Data, 
    Regiao, 
    Pais, 
    Latitude, 
    Longitude,
    ForecastDay.day.maxtemp_c as TemperaturaMaxima_Celsius,
    ForecastDay.day.mintemp_c as TemperaturaMinima_Celsius,
    ForecastDay.day.avgtemp_c as TemperaturaMedia_Celsius,
    CASE 
        WHEN ForecastDay.day.daily_will_it_rain == 1 THEN 'Sim'
        ELSE 'Nao' 
    END as VaiChover,
    ForecastDay.day.daily_chance_of_rain as ChanceDeChuva,
    ForecastDay.day.condition.text as CondicaoDoTempo,
    ForecastDay.astro.sunrise as NascerDoSol_LocalTime,
    ForecastDay.astro.sunset as PorDoSol_LocalTime,
    ForecastDay.day.maxwind_kph as VelocidadeMaximaDoVento_kph

    FROM
      (SELECT 
        explode(forecast.forecastday) AS ForecastDay,
        id as CidadeID,
        location.name as Cidade,
        location.region as Regiao,
        location.country as Pais,
        location.lat as Latitude,
        location.lon as Longitude
          FROM 
            forecastsView INNER JOIN citiesView 
            ON citiesView.nome == forecastsView.location.name)

    ORDER BY Cidade ASC
  """)

DF_tabela1.show(10,truncate=False)

+---------+--------+----------+---------+------+--------+---------+-------------------------+-------------------------+------------------------+---------+-------------+--------------------+---------------------+------------------+---------------------------+
|Cidade   |CidadeID|Data      |Regiao   |Pais  |Latitude|Longitude|TemperaturaMaxima_Celsius|TemperaturaMinima_Celsius|TemperaturaMedia_Celsius|VaiChover|ChanceDeChuva|CondicaoDoTempo     |NascerDoSol_LocalTime|PorDoSol_LocalTime|VelocidadeMaximaDoVento_kph|
+---------+--------+----------+---------+------+--------+---------+-------------------------+-------------------------+------------------------+---------+-------------+--------------------+---------------------+------------------+---------------------------+
|Aparecida|3502507 |2022-05-08|Sao Paulo|Brazil|-22.83  |-45.23   |23.6                     |12.8                     |16.9                    |Sim      |79           |Patchy rain possible|06:23 AM             |05:31 PM    

In [417]:
# Criar DF da Tabela 2
DF_tabela2 = spark.sql("""
  SELECT
    Cidade, 
    SUM(ForecastDay.day.daily_will_it_rain) AS QtdDiasVaiChover, 
    (
      COUNT(ForecastDay.date) - SUM(ForecastDay.day.daily_will_it_rain)
    ) AS QtdDiasNaoVaiChover,
    COUNT(ForecastDay.date) AS TotalDiasMapeados  

    FROM
      (SELECT 
        explode(forecast.forecastday) AS ForecastDay,
        location.name as Cidade
          FROM 
            forecastsView)
    GROUP BY Cidade       
    ORDER BY Cidade ASC
  """)

DF_tabela2.show(200,truncate=False)

+------------------------+----------------+-------------------+-----------------+
|Cidade                  |QtdDiasVaiChover|QtdDiasNaoVaiChover|TotalDiasMapeados|
+------------------------+----------------+-------------------+-----------------+
|Aparecida               |1               |2                  |3                |
|Arapei                  |3               |0                  |3                |
|Areias                  |3               |0                  |3                |
|Bananal                 |3               |0                  |3                |
|Barreiro                |0               |3                  |3                |
|Cacapava                |1               |2                  |3                |
|Cachoeira Paulista      |1               |2                  |3                |
|Campos                  |0               |3                  |3                |
|Canas                   |3               |0                  |3                |
|Caraguatatuba  

In [418]:
# Exportar CSVs

DF_tabela1.coalesce(1).write.csv(
    path = '/content/csvtabela1',
    mode = 'overwrite',
    sep = ';',
    header=True
)

DF_tabela2.coalesce(1).write.csv(
    path = '/content/csvtabela2',
    mode = 'overwrite',
    sep = ';',
    header=True
)